## Compare the trained models
Load the checkpoints, get the test datamodel up and running, access the two channel output.

In [1]:
import torch
from mimo.models.mimo_unet import MimoUnetModel
from mimo.tasks.sen12tp.sen12tp_datamodule import get_datamodule

In [21]:
# defining args within the notebook
args = {
    'seed': 1,
    'checkpoint_path': 'MIMO_NDVI_Prediction_Gauss/1yt2l40t/checkpoints/last.ckpt',
    'dataset_dir': '/deepskieslab/rnevin/zenodo_data',
    'batch_size': 32,
    'num_loss_function_params': 2,
    'num_subnetworks': 2,
    'filter_base_count': 30,
    'center_dropout_rate': 0.1,
    'final_dropout_rate': 0.1,
    'encoder_dropout_rate': 0.0,
    'core_dropout_rate': 0.0,
    'decoder_dropout_rate': 0.0,
    'loss_buffer_size': 10,
    'loss_buffer_temperature': 0.3,
    'input_repetition_probability': 0.0,
    'batch_repetitions': 1,
    'patch_size': 256,
    'stride': 249,
    'loss': 'gaussian_nll',  # Adjust based on your actual loss function
    'weight_decay': 0.0001,
    'learning_rate': 0.0001,
    'num_workers': 30,
    'training_set_percentage': 1.0,
}

args["input"] = ["VV_sigma0", "VH_sigma0"]
args["target"] = ["NDVI"]  # Example target

'''
python scripts/train/train_ndvi.py   --max_epochs 40   --batch_size 32   -t NDVI   -i VV_sigma0   -i VH_sigma0   --project "MIMO_NDVI_Prediction_Gauss"
'''

from argparse import Namespace

args = Namespace(**args)  # Convert dictionary to Namespace
dm = get_datamodule(args)


# Initialize the datamodule (this can be modified based on your setup)
dm = get_datamodule(args)

# Instantiate the model (ensure to use the same parameters as during training)
model = MimoUnetModel(
    in_channels=len(dm.model_inputs),
    out_channels=len(dm.model_targets) * args.num_loss_function_params,
    num_subnetworks=args.num_subnetworks,
    filter_base_count=args.filter_base_count,
    center_dropout_rate=args.center_dropout_rate,
    final_dropout_rate=args.final_dropout_rate,
    encoder_dropout_rate=args.encoder_dropout_rate,
    core_dropout_rate=args.core_dropout_rate,
    decoder_dropout_rate=args.decoder_dropout_rate,
    loss_buffer_size=args.loss_buffer_size,
    loss_buffer_temperature=args.loss_buffer_temperature,
    input_repetition_probability=args.input_repetition_probability,
    batch_repetitions=args.batch_repetitions,
    loss=args.loss,
    weight_decay=args.weight_decay,
    learning_rate=args.learning_rate,
    seed=args.seed,
)


In [22]:
print(args)
# Load the checkpoint
checkpoint = torch.load(args.checkpoint_path)
print(checkpoint.keys())
model.load_state_dict(checkpoint['state_dict'])

# Set the model to evaluation mode
model.eval()

print(model)

Namespace(seed=1, checkpoint_path='MIMO_NDVI_Prediction_Gauss/1yt2l40t/checkpoints/last.ckpt', dataset_dir='/deepskieslab/rnevin/zenodo_data', batch_size=32, num_loss_function_params=2, num_subnetworks=2, filter_base_count=30, center_dropout_rate=0.1, final_dropout_rate=0.1, encoder_dropout_rate=0.0, core_dropout_rate=0.0, decoder_dropout_rate=0.0, loss_buffer_size=10, loss_buffer_temperature=0.3, input_repetition_probability=0.0, batch_repetitions=1, patch_size=256, stride=249, loss='gaussian_nll', weight_decay=0.0001, learning_rate=0.0001, num_workers=30, training_set_percentage=1.0, input=['VV_sigma0', 'VH_sigma0'], target=['NDVI'])


/tmp/ipykernel_6550/63106582.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.checkpoint_path)


dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'MixedPrecision', 'hparams_name', 'hyper_parameters'])


RuntimeError: Error(s) in loading state_dict for MimoUnetModel:
	Missing key(s) in state_dict: "model.encoder.in_convs.0.double_conv.0.weight", "model.encoder.in_convs.0.double_conv.0.bias", "model.encoder.in_convs.0.double_conv.1.weight", "model.encoder.in_convs.0.double_conv.1.bias", "model.encoder.in_convs.0.double_conv.1.running_mean", "model.encoder.in_convs.0.double_conv.1.running_var", "model.encoder.in_convs.0.double_conv.3.weight", "model.encoder.in_convs.0.double_conv.3.bias", "model.encoder.in_convs.0.double_conv.4.weight", "model.encoder.in_convs.0.double_conv.4.bias", "model.encoder.in_convs.0.double_conv.4.running_mean", "model.encoder.in_convs.0.double_conv.4.running_var", "model.encoder.in_convs.1.double_conv.0.weight", "model.encoder.in_convs.1.double_conv.0.bias", "model.encoder.in_convs.1.double_conv.1.weight", "model.encoder.in_convs.1.double_conv.1.bias", "model.encoder.in_convs.1.double_conv.1.running_mean", "model.encoder.in_convs.1.double_conv.1.running_var", "m

In [ ]:

# Now we can use the model to make predictions
test_data = dm.test_dataloader()  # Ensure the dataloader is set up for your test dataset

# Make predictions
predictions = []
with torch.no_grad():
    for inputs in test_data:
        outputs = model(inputs)
        predictions.append(outputs)

# Optionally save the predictions (or process them further)
for i, pred in enumerate(predictions):
    save_image(pred, f'prediction_{i}.png')  # Replace with actual saving function if needed



In [ ]:
from mimo.models.mimo_unet import MimoUnetModel

In [ ]:
import torch

# Assuming your model is defined as `model` and the checkpoint is saved as 'checkpoint.pth'
checkpoint_path = 'MIMO_NDVI_Prediction_Gauss/1yt2l40t/checkpoints/last.ckpt'

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)

# Assuming the checkpoint contains 'model_state_dict' for the model weights
model.load_state_dict(checkpoint['model_state_dict'])

# Set the model to evaluation mode
model.eval()